In [1]:
# Import needed libraries and load API keys from .env file

import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

from langchain.document_loaders.base import Document
from langchain.utilities import ApifyWrapper

In [3]:
# Instantiate Apify

apify = ApifyWrapper()

In [24]:
# Define the input for the crawler. 
# See the "Input" section of the Website Content Crawler on 
# https://console.apify.com/actors for configuration options.

crawl_input={
    "htmlTransformer": "extractus",
    "crawlerType": "cheerio",
    "startUrls": [{"url": "https://www.rand.org/health-care/about.html"}]
}

In [25]:
loader = apify.call_actor(
    actor_id='apify/website-content-crawler', 
    run_input=crawl_input, 
    dataset_mapping_function=lambda item: Document(
        page_content=item['text'] or '',
        metadata={'source': item['url']}
    )
)

In [26]:
# Inspect the extracted docs
docs = loader.load()

In [27]:
docs

[Document(page_content="Skip to section navigation\nSkip to page content\nObjective Analysis. Effective Solutions.\nHealth Care\nToggle Menu \nToggle Search \nSite-wide SearchSearch\nAbout RAND Health Care\nRAND researchers Sangeeta Ahluwalia and Sarah MacCarthy participate in a panel for RAND's Summer Associate Program.\nPhoto by Diane Baldwin\nContact\nRANDHealthCare@rand.org \nRAND Health Care\n1776 Main Street\nP.O. Box 2138\nSanta Monica, CA 90407-2138\nWho We Are\nRAND Health Care is a research division of the RAND Corporation, a nonprofit institution that helps improve policy and decisionmaking through research and analysis.\nOur Mission\nRAND Health Care promotes healthier societies by improving health care systems in the United States and other countries. We provide health care decisionmakers, practitioners, and the public with actionable, rigorous, objective evidence to support their most complex decisions.\nOur Capabilities\nRAND Health Care possesses broad, deep, and longst

In [21]:
# Instantiate GPT-4 and define load_qa chain
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

chat = ChatOpenAI()
chain = load_qa_chain(chat, chain_type="stuff")

In [28]:
# Your main query to the language model - modify to you needs
q="""
Formulate a short 5 line email to RAND Health Care pithcing Arrakis product below. The email should 
make a reference to RAND's work and give them a compliment.

Arrakis
The Mission: We builds data and software tools for sales and marketing teams that sell healthcare products, to ensure the latest in healthcare innovation reaches the right providers and patients faster.

The Product: We've built a database of the US Healthcare System and help our customers identify the people and organizations most likely to need their services. Now we're also helping them automate highly personalized outreach.

The Team: Previously Sam (CEO) co-founded a clinical testing lab which raised $14M+ in VC led by A16z, out of which spun off a COVID testing lab that did 36 million COVID tests. Sav (CTO) scaled an EdTech start-up to $50M in Revenue as the CTO. Founding Engineer Chris was a data engineer at Stripe, and Rayhnuma (Head of Biz Ops) was early at 2 acquired startups.

The Traction: We’ve raised $2M+, built an product, and have hundreds of thousands of dollars in revenue."""

In [29]:
# The personalized email or paragraph you can send to Lemlist
email = chain.run(input_documents=docs, question=q)
print(email)

Subject: Collaboration Opportunity with Arrakis: Leveraging RAND's Expertise in Healthcare Innovation

Dear RAND Health Care Team,

I hope this email finds you well. I wanted to reach out to you because I have been following RAND's invaluable work in improving health care systems and decision-making through rigorous research and analysis. Your dedication to promoting healthier societies aligns perfectly with the mission of our company, Arrakis.

At Arrakis, we have developed a comprehensive database of the US Healthcare System, enabling our customers to identify the individuals and organizations most likely to benefit from their healthcare products and services. We are also proud to offer automated, highly personalized outreach to streamline the process of reaching the right providers and patients faster.

Our team, comprised of industry experts with a track record of success, has raised over $2 million in funding, built a robust product, and generated significant revenue. We believe t